In [1]:
import pandas as pd 
import numpy as np 
import re 
from nltk.corpus import stopwords 
from nltk.tokenize import TweetTokenizer 
from nltk.stem.wordnet import WordNetLemmatizer 
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer 
from sklearn.linear_model import LogisticRegression 
from sklearn.svm import SVC 
from sklearn.svm import LinearSVC 
from sklearn.calibration import CalibratedClassifierCV 
from sklearn.metrics import roc_auc_score 
from scipy.sparse import csr_matrix, hstack 
import lightgbm as lgb

In [2]:
PATH = '../data/'

train = pd.read_csv(PATH + 'cleaned_train.csv')
test = pd.read_csv(PATH + 'cleaned_test.csv')


train_sentence = train['comment_text_cleaned_polarity']
test_sentence = test['comment_text_cleaned_polarity']


train_sentence_retain_punctuation = train['comment_text_cleaned_retain_punctuation']
test_sentence_retain_punctuation = test['comment_text_cleaned_retain_punctuation']

text = train_sentence

text_retain_punctuation = train_sentence_retain_punctuation


print(train.shape)
print(test.shape)

(159571, 30)
(153164, 24)


In [3]:


phrase_vectorizer = TfidfVectorizer(ngram_range=(1,3),
                                    strip_accents='unicode', 
                                    max_features=100000, 
                                    analyzer='word',
                                    sublinear_tf=True,
                                    token_pattern=r'\w{1,}')
char_vectorizer = TfidfVectorizer(ngram_range=(2,5), 
                                  strip_accents='unicode', 
                                  max_features=200000, 
                                  analyzer='char', 
                                  sublinear_tf=True)

print('fitting char')
char_vectorizer.fit(text_retain_punctuation.values)
print('fitting phrase')
phrase_vectorizer.fit(text.values)

print('transforming train skip gram')

print('transforming train char')
train_char = char_vectorizer.transform(train_sentence_retain_punctuation.values)
print('transforming train phrase')
train_phrase = phrase_vectorizer.transform(train_sentence.values)


print('transforming test char')
test_char = char_vectorizer.transform(test_sentence_retain_punctuation.values)
print('transforming test phrase')
test_phrase = phrase_vectorizer.transform(test_sentence.values)


train_tfidf = hstack((train_char, train_phrase), format='csr')
test_tfidf = hstack((test_char, test_phrase), format='csr')

label_cols = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

train_tfidf

fitting char
fitting phrase
transforming train skip gram
transforming train char
transforming train phrase
transforming test char
transforming test phrase


<159571x300000 sparse matrix of type '<class 'numpy.float64'>'
	with 162813546 stored elements in Compressed Sparse Row format>

In [4]:
# from sklearn.model_selection import train_test_split
# x_train, x_val, y_train_df, y_val_df = train_test_split(train_tfidf, train, test_size=0.33)
# # Split the dataset



# Split the dataset
split_index = round(len(train) * 0.8) #################################
# shuffled_train = train#.sample(frac=1)
x_train = train_tfidf[:split_index]
y_train_df = train.iloc[:split_index]
#######
x_val = train_tfidf[split_index:]
y_val_df = train.iloc[split_index:]
# Get test data ready
x_test = test_tfidf


# train toxic
def pr(y_i, y, train_features):
    p = train_features[y==y_i].sum(0)
    return (p + 1) / ((y == y_i).sum() + 1)
r_dict = {label: np.log(pr(1, y_train_df[label].values, x_train) / pr(0,  y_train_df[label].values, x_train)) for label in label_cols}

In [5]:
train_set = {label: x_train.multiply(r_dict[label]).tocsr() for label in r_dict }
val_set = {label: x_val.multiply(r_dict[label]).tocsr() for label in r_dict }
test_set = {label: x_test.multiply(r_dict[label]).tocsr() for label in r_dict }

# del r_dict, x_train, x_val
import gc
gc.collect()


278

In [ ]:
del r_dict, x_train, x_val
import gc
gc.collect()

In [ ]:
for 
lgb_train_set = lgb.Dataset(train_set['toxic'], y_train_df['toxic'].values)
lgb_eval_set = lgb.Dataset(val_set['toxic'], y_val_df['toxic'].values, reference=lgb_train_set)


# params = {
#     'task': 'train',
#     'boosting_type': 'gbdt',
#     'objective': 'binary',
#     'metric': {'binary', 'auc'},
#     'learning_rate': 0.2,
#     'num_iterations': 100,
#     'num_leaves': 171,
#     'device': 'cpu',
#     'num_threads': 24,
#     'max_depth': -1,
# #     'min_data_in_leaf': 5,
#     'bagging_fraction': 0.8,
#     'bagging_freq': 1,
#     'feature_fraction': 0.8,
#     'lambda_l1': 1,
#     'lambda_l2': 1}


params = {
    'learning_rate': 0.2,
    'application': 'binary',
    'num_leaves': 31,
    'verbosity': -1,
    'metric': 'auc',
    'data_random_seed': 2,
    'bagging_fraction': 0.8,
    'feature_fraction': 0.6,
    'nthread': 4,
    'lambda_l1': 1,
    'lambda_l2': 1
} 

gbm = lgb.train(params,
                lgb_train_set,
                valid_sets=lgb_eval_set,
               verbose_eval=10)



In [ ]:
from sklearn.metrics import roc_auc_score
y = y_val_df['toxic']
pred = gbm.predict(val_set['toxic'])
print('accuracy is {}'.format(roc_auc_score(y,pred)))

In [ ]:
inv_char = {v: k for k, v in char_vectorizer.vocabulary_.items()}
inv_phrase = {v: k for k, v in phrase_vectorizer.vocabulary_.items()}
features = []
features_char = [inv_char[i] for i in range(0,train_char.shape[1] )]
features_phrase = [inv_phrase[i] for i in range(0,train_phrase.shape[1])]
features.extend(features_char)
features.extend(features_phrase)

In [ ]:
df_importance = pd.DataFrame()
df_importance['score'] = pd.Series(gbm.feature_importance()) 
df_importance['feature'] = pd.Series(features)

In [ ]:
df_importance.sort_values(by=['score'], ascending=False)

In [ ]:
cols = {'combination_id': [], 'learning_rate':[], 'num_leaves':[], 
        'bagging_fraction':[], 'feature_fraction':[], 'lambda_l1':[], 
        'lambda_l2':[], 'label':[],'roc':[],'trained':[], 'max_depth': [], 
        'num_iterations': [], 'tp': [], 'fp': [], 'precision': []}
df_lgb = pd.DataFrame(cols)
learning_rate = [ 0.2, 0.1, 0.05]
num_leaves = [11,31, 61,]
max_depth = [3,4,5]
bag_frt = [0.9, 0.8,0.7]
feature_fraction = [1, 0.8,0.6]
lambda_l1 = [1, 0, 0.5]
lambda_l2 = [1, 0, 0.5]
num_iterations = [8000]
is_unbalance = [True, False]

com_id = 0
index = 0
for col in label_cols:
    print(col)
    com_id += 1
    for lr in learning_rate:
        for nl in num_leaves:
            for md in max_depth:
                for bf in bag_frt:
                    for ff in feature_fraction:
                        for l1 in lambda_l1:
                            for l2 in lambda_l2:
                                for ni in num_iterations:
                                    for isb in is_unbalance:
                                        df_lgb.loc[index, 'is_unbalance'] = str(isb)
                                        df_lgb.loc[index, 'num_iterations'] = ni
                                        df_lgb.loc[index, 'lambda_l2'] = l2
                                        df_lgb.loc[index, 'lambda_l1'] = l1
                                        df_lgb.loc[index, 'feature_fraction'] = ff
                                        df_lgb.loc[index, 'bagging_fraction'] = bf
                                        df_lgb.loc[index, 'max_depth'] = md
                                        df_lgb.loc[index, 'num_leaves'] = nl
                                        df_lgb.loc[index, 'learning_rate'] = lr
                                        df_lgb.loc[index, 'label'] = col
                                        df_lgb.loc[index, 'roc'] = 0
                                        df_lgb.loc[index, 'tp'] = 0
                                        df_lgb.loc[index, 'fp'] = 0
                                        df_lgb.loc[index, 'precision'] = 0
                                        df_lgb.loc[index, 'trained'] = 'N'
                                        df_lgb.loc[index, 'combination_id'] = com_id
                                        index += 1
df_lgb.to_csv(PATH + 'lgb_all_gridsearch.csv', index=False)                        

In [12]:
df_lgb = pd.read_csv(PATH + 'lgb_all_gridsearch.csv')

In [13]:
# df_lgb.shape

(43740, 17)

In [14]:
base_params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': {'binary', 'auc'},
    'verbosity': 20,
    'verbose': 20,
    'metric': 'auc',
    'bagging_freq': 1,
    'num_threads': 8,
    'early_stopping_rounds':20,
    'verbose_eval':10
    } 
def get_parameters(df_lgb , col, addition = None):
    params_list =['bagging_fraction',  
                     'feature_fraction', 
                     'lambda_l1', 
                     'lambda_l2', 
                     'learning_rate', 
                     'max_depth',
                     'num_iterations', 
                     'num_leaves',
                     'is_unbalance']
    condition = (df_lgb['trained'] == 'N') & (df_lgb['label'] == col)
    if addition is not None:
        for i in range(len(addition)):
            condition = condition &  (df_lgb[addition[i]['key']] == addition[i]['value'])
#     df = df_lgb[(df_lgb['trained'] == 'N') & (df_lgb['label'] == col)]
    df = df_lgb[condition]
    index = df.index.values
    np.random.shuffle(index)
    next_index = index[0]
    params = dict(df_lgb.loc[next_index, params_list])
    ##### data type fix:
    params['num_leaves'] = int(round(params['num_leaves']))
    params['max_depth'] = int(round(params['max_depth']))
    params['num_iterations'] = int(round(params['num_iterations']))
    params['is_unbalance'] = bool(params['is_unbalance'])
    return params, next_index
    
    
    

In [ ]:
################# random search start
from sklearn.metrics import roc_auc_score, confusion_matrix

%env JOBLIB_TEMP_FOLDER=/tmp
start = 0
for i in range(10000):
    for col in label_cols:
        start += 1
        # create dataset for lightgbm
        params =  base_params.copy()  
        add = [{'key':'num_iterations', 'value': 1200}]
        add = None
        params_get, index_get = get_parameters(df_lgb, col, addition=add)
        params.update(params_get)
        print(start)
        print(col)
        print(index_get)
        print(params)
        ############################################
        gbm_model = lgb.LGBMClassifier(**params)
        gbm_model.fit(train_set[col], y_train_df[col].values,
                      eval_set=[(val_set[col], y_val_df[col].values)], 
                      eval_metric =['logloss', 'auc'],
                      early_stopping_rounds=params['early_stopping_rounds'])
        pred_prob =  gbm_model.predict_proba(val_set[col])[:,1]
        pred = gbm_model.predict(val_set[col])
        roc = roc_auc_score(y_val_df[col].values,pred_prob)
        tn, fp, fn, tp = confusion_matrix(y_val_df[col].values, pred).ravel()
        precision = tp / (tp + fp)
        tpr = tp / (tp + fn)
        fpr = fp / (fp + tn)

        df_lgb.loc[index_get, 'precision'] = precision
        df_lgb.loc[index_get, 'tp'] = tpr
        df_lgb.loc[index_get, 'fp'] = fpr
        df_lgb.loc[index_get, 'roc'] = roc
        df_lgb.loc[index_get, 'trained'] = 'Y'
        df_lgb.loc[index_get, 'roc'] = roc
        df_lgb.loc[index_get, 'best_round'] = gbm_model.best_iteration_
        print('ROC score is {}'.format(roc) )
        df_lgb.to_csv(PATH + 'lgb_all_gridsearch.csv', index=False) 
        print('done')
        print('===================================')
        
        ################################################
#         lgb_train_set = lgb.Dataset(train_set[col], y_train_df[col].values)
#         lgb_eval_set = lgb.Dataset(val_set[col], y_val_df[col].values, reference=lgb_train_set)


#         gbm_model = lgb.train(params,
#                             lgb_train_set,
#                             valid_sets=lgb_eval_set,
#                            verbose_eval=20)
#         pred_prob =  gbm_model.predict(val_set[col])
#         roc = roc_auc_score(y_val_df[col].values,pred_prob)
#         df_lgb.loc[index_get, 'trained'] = 'Y'
#         df_lgb.loc[index_get, 'roc'] = roc
#         print('ROC score is {}'.format(roc) )
#         df_lgb.to_csv(PATH + 'lgb_all_gridsearch.csv', index=False) 
#         print('done')
#         print('===================================')
    

env: JOBLIB_TEMP_FOLDER=/tmp
1
toxic
27642
{'task': 'train', 'boosting_type': 'gbdt', 'objective': 'binary', 'metric': 'auc', 'verbosity': 20, 'verbose': 20, 'bagging_freq': 1, 'num_threads': 8, 'early_stopping_rounds': 20, 'verbose_eval': 10, 'bagging_fraction': 0.80000000000000004, 'feature_fraction': 0.59999999999999998, 'lambda_l1': 0.5, 'lambda_l2': 1.0, 'learning_rate': 0.050000000000000003, 'max_depth': 6, 'num_iterations': 8000, 'num_leaves': 61, 'is_unbalance': True}


/home/kai/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:99: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/home/kai/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:104: UserWarning: Found `early_stopping_rounds` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[1]	valid_0's auc: 0.844299
Training until validation scores don't improve for 20 rounds.
[2]	valid_0's auc: 0.87195
[3]	valid_0's auc: 0.873944
[4]	valid_0's auc: 0.878321
[5]	valid_0's auc: 0.879384
[6]	valid_0's auc: 0.880798
[7]	valid_0's auc: 0.88376
[8]	valid_0's auc: 0.885138
[9]	valid_0's auc: 0.885596
[10]	valid_0's auc: 0.889052
[11]	valid_0's auc: 0.889749
[12]	valid_0's auc: 0.891295
[13]	valid_0's auc: 0.894161
[14]	valid_0's auc: 0.895464
[15]	valid_0's auc: 0.895786
[16]	valid_0's auc: 0.89705
[17]	valid_0's auc: 0.898427
[18]	valid_0's auc: 0.899225
[19]	valid_0's auc: 0.900427
[20]	valid_0's auc: 0.90238
[21]	valid_0's auc: 0.903243
[22]	valid_0's auc: 0.904329
[23]	valid_0's auc: 0.90524
[24]	valid_0's auc: 0.906017
[25]	valid_0's auc: 0.907231
[26]	valid_0's auc: 0.908755
[27]	valid_0's auc: 0.909578
[28]	valid_0's auc: 0.910371
[29]	valid_0's auc: 0.910887
[30]	valid_0's auc: 0.911795
[31]	valid_0's auc: 0.912663
[32]	valid_0's auc: 0.9142
[33]	valid_0's auc: 0.9149

[277]	valid_0's auc: 0.962869
[278]	valid_0's auc: 0.962927
[279]	valid_0's auc: 0.962958
[280]	valid_0's auc: 0.963024
[281]	valid_0's auc: 0.963079
[282]	valid_0's auc: 0.963102
[283]	valid_0's auc: 0.963126
[284]	valid_0's auc: 0.963156
[285]	valid_0's auc: 0.963196
[286]	valid_0's auc: 0.963222
[287]	valid_0's auc: 0.96325
[288]	valid_0's auc: 0.963246
[289]	valid_0's auc: 0.963323
[290]	valid_0's auc: 0.963347
[291]	valid_0's auc: 0.963367
[292]	valid_0's auc: 0.963406
[293]	valid_0's auc: 0.963379
[294]	valid_0's auc: 0.963445
[295]	valid_0's auc: 0.963465
[296]	valid_0's auc: 0.963497
[297]	valid_0's auc: 0.96354
[298]	valid_0's auc: 0.963581
[299]	valid_0's auc: 0.963601
[300]	valid_0's auc: 0.963669
[301]	valid_0's auc: 0.963712
[302]	valid_0's auc: 0.963756
[303]	valid_0's auc: 0.963772
[304]	valid_0's auc: 0.963802
[305]	valid_0's auc: 0.963847
[306]	valid_0's auc: 0.963877
[307]	valid_0's auc: 0.963905
[308]	valid_0's auc: 0.963935
[309]	valid_0's auc: 0.963963
[310]	valid_

[551]	valid_0's auc: 0.969166
[552]	valid_0's auc: 0.969171
[553]	valid_0's auc: 0.969186
[554]	valid_0's auc: 0.969198
[555]	valid_0's auc: 0.969223
[556]	valid_0's auc: 0.969232
[557]	valid_0's auc: 0.969256
[558]	valid_0's auc: 0.969264
[559]	valid_0's auc: 0.969279
[560]	valid_0's auc: 0.969303
[561]	valid_0's auc: 0.969334
[562]	valid_0's auc: 0.969335
[563]	valid_0's auc: 0.969379
[564]	valid_0's auc: 0.969378
[565]	valid_0's auc: 0.969387
[566]	valid_0's auc: 0.969381
[567]	valid_0's auc: 0.969389
[568]	valid_0's auc: 0.969386
[569]	valid_0's auc: 0.969399
[570]	valid_0's auc: 0.969401
[571]	valid_0's auc: 0.969427
[572]	valid_0's auc: 0.969445
[573]	valid_0's auc: 0.969478
[574]	valid_0's auc: 0.969503
[575]	valid_0's auc: 0.969523
[576]	valid_0's auc: 0.96956
[577]	valid_0's auc: 0.969557
[578]	valid_0's auc: 0.969604
[579]	valid_0's auc: 0.969616
[580]	valid_0's auc: 0.96963
[581]	valid_0's auc: 0.969628
[582]	valid_0's auc: 0.969636
[583]	valid_0's auc: 0.969658
[584]	valid_

...

[277]	valid_0's auc: 0.977292
[278]	valid_0's auc: 0.977323
[279]	valid_0's auc: 0.977344
[280]	valid_0's auc: 0.977384
[281]	valid_0's auc: 0.977387
[282]	valid_0's auc: 0.97744
[283]	valid_0's auc: 0.977463
[284]	valid_0's auc: 0.977396
[285]	valid_0's auc: 0.977385
[286]	valid_0's auc: 0.977434
[287]	valid_0's auc: 0.97748
[288]	valid_0's auc: 0.977518
[289]	valid_0's auc: 0.977483
[290]	valid_0's auc: 0.977488
[291]	valid_0's auc: 0.97752
[292]	valid_0's auc: 0.977541
[293]	valid_0's auc: 0.977564
[294]	valid_0's auc: 0.977576
[295]	valid_0's auc: 0.977584
[296]	valid_0's auc: 0.977564
[297]	valid_0's auc: 0.977543
[298]	valid_0's auc: 0.977548
[299]	valid_0's auc: 0.97756
[300]	valid_0's auc: 0.977567
[301]	valid_0's auc: 0.977595
[302]	valid_0's auc: 0.977675
[303]	valid_0's auc: 0.977684
[304]	valid_0's auc: 0.977698
[305]	valid_0's auc: 0.977729
[306]	valid_0's auc: 0.977661
[307]	valid_0's auc: 0.977668
[308]	valid_0's auc: 0.977695
[309]	valid_0's auc: 0.977676
[310]	valid_0'

/home/kai/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:99: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/home/kai/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:104: UserWarning: Found `early_stopping_rounds` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[1]	valid_0's auc: 0.799427
Training until validation scores don't improve for 20 rounds.
[2]	valid_0's auc: 0.887906
[3]	valid_0's auc: 0.929265
[4]	valid_0's auc: 0.930212
[5]	valid_0's auc: 0.938573
[6]	valid_0's auc: 0.943514
[7]	valid_0's auc: 0.950016
[8]	valid_0's auc: 0.956112
[9]	valid_0's auc: 0.960442
[10]	valid_0's auc: 0.962647
[11]	valid_0's auc: 0.96447
[12]	valid_0's auc: 0.966677
[13]	valid_0's auc: 0.967838
[14]	valid_0's auc: 0.967065
[15]	valid_0's auc: 0.967789
[16]	valid_0's auc: 0.970163
[17]	valid_0's auc: 0.971641
[18]	valid_0's auc: 0.97324
[19]	valid_0's auc: 0.973568
[20]	valid_0's auc: 0.972216
[21]	valid_0's auc: 0.973084
[22]	valid_0's auc: 0.973871
[23]	valid_0's auc: 0.973368
[24]	valid_0's auc: 0.973329
[25]	valid_0's auc: 0.972943
[26]	valid_0's auc: 0.973258
[27]	valid_0's auc: 0.973082
[28]	valid_0's auc: 0.972464
[29]	valid_0's auc: 0.972788
[30]	valid_0's auc: 0.973024
[31]	valid_0's auc: 0.972566
[32]	valid_0's auc: 0.972761
[33]	valid_0's auc: 0

/home/kai/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:99: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/home/kai/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:104: UserWarning: Found `early_stopping_rounds` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[1]	valid_0's auc: 0.839577
Training until validation scores don't improve for 20 rounds.
[2]	valid_0's auc: 0.849296
[3]	valid_0's auc: 0.856683
[4]	valid_0's auc: 0.858661
[5]	valid_0's auc: 0.861429
[6]	valid_0's auc: 0.868627
[7]	valid_0's auc: 0.871909
[8]	valid_0's auc: 0.87329
[9]	valid_0's auc: 0.877114
[10]	valid_0's auc: 0.87968
[11]	valid_0's auc: 0.880475
[12]	valid_0's auc: 0.882178
[13]	valid_0's auc: 0.883357
[14]	valid_0's auc: 0.886194
[15]	valid_0's auc: 0.887907
[16]	valid_0's auc: 0.889782
[17]	valid_0's auc: 0.891737
[18]	valid_0's auc: 0.892867
[19]	valid_0's auc: 0.893652
[20]	valid_0's auc: 0.894392
[21]	valid_0's auc: 0.896129
[22]	valid_0's auc: 0.897339
[23]	valid_0's auc: 0.898224
[24]	valid_0's auc: 0.899345
[25]	valid_0's auc: 0.901045
[26]	valid_0's auc: 0.901755
[27]	valid_0's auc: 0.902347
[28]	valid_0's auc: 0.902977
[29]	valid_0's auc: 0.904447
[30]	valid_0's auc: 0.905086
[31]	valid_0's auc: 0.905667
[32]	valid_0's auc: 0.906413
[33]	valid_0's auc: 0

In [ ]:
df_lgb.to_csv(PATH + 'lgb_all_gridsearch.csv', index=False) 
print('done')

In [ ]:
##############################
lgb_train_set = {}
lgb_eval_set = {}
gbm_model = {}
y_pred = pd.DataFrame()
y_pred['id'] = test['id']
for col in label_cols:
    # create dataset for lightgbm


    params = {
    'learning_rate': 0.2,
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': {'binary', 'auc'},
    'num_leaves': 31,
    'verbosity': -1,
    'metric': 'auc',
    'data_random_seed': 2,
    'bagging_fraction': 0.8,
    'feature_fraction': 0.6,
    'bagging_freq': 1,
    'nthread': 12,
    'lambda_l1': 1,
    'lambda_l2': 1,
    'early_stopping_round':10
    } 

    print(col)
    lgb_train_set[col] = lgb.Dataset(train_set[col], y_train_df[col].values)
    lgb_eval_set[col] = lgb.Dataset(train_set[col], y_train_df[col].values, reference=lgb_train_set[col])
    

    gbm_model[col] = lgb.train(params,
                        lgb_train_set[col],
                        valid_sets=lgb_eval_set[col],
                       verbose_eval=20)
    y_pred[col] =  gbm_model[col].predict(test_set[col])

In [6]:
df_lgb = pd.read_csv(PATH + 'lgb_all_gridsearch.csv') 

In [7]:
def get_parameters_best(df_lgb , col, addition = None):
    params_list =['bagging_fraction',  
                     'feature_fraction', 
                     'lambda_l1', 
                     'lambda_l2', 
                     'learning_rate', 
                     'max_depth',
                     'num_iterations', 
                     'num_leaves',
                     'is_unbalance']
    condition = (df_lgb['trained'] == 'Y') & (df_lgb['label'] == col)
    if addition is not None:
        for i in range(len(addition)):
            condition = condition &  (df_lgb[addition[i]['key']] == addition[i]['value'])
#     df = df_lgb[(df_lgb['trained'] == 'N') & (df_lgb['label'] == col)]
    df = df_lgb[condition].sort_values(['roc'], ascending=False)
#     print(df)
    index = df.index.values[0]
    next_index = index
#     print(df.loc[next_index])
    params = dict(df_lgb.loc[next_index, params_list])
    ##### data type fix:
    params['num_leaves'] = int(round(params['num_leaves']))
    params['max_depth'] = int(round(params['max_depth']))
    params['num_iterations'] = int(round(params['num_iterations']))
    params['is_unbalance'] = bool(params['is_unbalance'])
    return params, next_index

In [9]:
base_params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': {'binary', 'auc'},
    'verbosity': 20,
    'verbose': 20,
    'metric': 'auc',
    'bagging_freq': 1,
    'num_threads': 8,
    'early_stopping_rounds':20,
    'verbose_eval':10
    } 
################# random search start
from sklearn.metrics import roc_auc_score, confusion_matrix
for col in label_cols:
    add= None
    params = base_params.copy()
    params_get, index_get = get_parameters_best(df_lgb, col, addition=add)
    params.update(params_get)
    print(index_get)
    gbm_model = lgb.LGBMClassifier(**params)
    gbm_model.fit(train_set[col], y_train_df[col].values,
                  eval_set=[(val_set[col], y_val_df[col].values)], 
                  eval_metric =['logloss', 'auc'],
                  early_stopping_rounds=params['early_stopping_rounds'])
    pred_prob =  gbm_model.predict_proba(val_set[col])[:,1]
    pred = gbm_model.predict(val_set[col])
    roc = roc_auc_score(y_val_df[col].values,pred_prob)
    tn, fp, fn, tp = confusion_matrix(y_val_df[col].values, pred).ravel()
    precision = tp / (tp + fp)
    tpr = tp / (tp + fn)
    fpr = fp / (fp + tn)

    df_lgb.loc[index_get, 'precision'] = precision
    df_lgb.loc[index_get, 'tp'] = tpr
    df_lgb.loc[index_get, 'fp'] = fpr
    df_lgb.loc[index_get, 'roc'] = roc
    df_lgb.loc[index_get, 'trained'] = 'Y'
    df_lgb.loc[index_get, 'roc'] = roc
    df_lgb.loc[index_get, 'best_round'] = gbm_model.best_iteration_
    df_lgb.to_csv(PATH + 'lgb_all_gridsearch.csv', index=False) 


3672


/home/kai/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:99: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/home/kai/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:104: UserWarning: Found `early_stopping_rounds` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[1]	valid_0's auc: 0.823776
Training until validation scores don't improve for 20 rounds.
[2]	valid_0's auc: 0.832574
[3]	valid_0's auc: 0.836026
[4]	valid_0's auc: 0.847304
[5]	valid_0's auc: 0.854497
[6]	valid_0's auc: 0.860009
[7]	valid_0's auc: 0.862395
[8]	valid_0's auc: 0.864164
[9]	valid_0's auc: 0.865686
[10]	valid_0's auc: 0.865955
[11]	valid_0's auc: 0.87057
[12]	valid_0's auc: 0.873179
[13]	valid_0's auc: 0.87508
[14]	valid_0's auc: 0.877163
[15]	valid_0's auc: 0.879009
[16]	valid_0's auc: 0.879842
[17]	valid_0's auc: 0.880722
[18]	valid_0's auc: 0.882833
[19]	valid_0's auc: 0.883727
[20]	valid_0's auc: 0.885906
[21]	valid_0's auc: 0.886594
[22]	valid_0's auc: 0.887771
[23]	valid_0's auc: 0.88784
[24]	valid_0's auc: 0.888501
[25]	valid_0's auc: 0.889575
[26]	valid_0's auc: 0.890609
[27]	valid_0's auc: 0.891558
[28]	valid_0's auc: 0.893251
[29]	valid_0's auc: 0.893926
[30]	valid_0's auc: 0.894859
[31]	valid_0's auc: 0.895479
[32]	valid_0's auc: 0.896584
[33]	valid_0's auc: 0.

[277]	valid_0's auc: 0.957086
[278]	valid_0's auc: 0.957192
[279]	valid_0's auc: 0.957242
[280]	valid_0's auc: 0.957311
[281]	valid_0's auc: 0.957395
[282]	valid_0's auc: 0.957401
[283]	valid_0's auc: 0.957424
[284]	valid_0's auc: 0.957432
[285]	valid_0's auc: 0.957477
[286]	valid_0's auc: 0.957547
[287]	valid_0's auc: 0.957619
[288]	valid_0's auc: 0.957663
[289]	valid_0's auc: 0.957719
[290]	valid_0's auc: 0.957786
[291]	valid_0's auc: 0.957801
[292]	valid_0's auc: 0.957842
[293]	valid_0's auc: 0.957895
[294]	valid_0's auc: 0.957943
[295]	valid_0's auc: 0.95799
[296]	valid_0's auc: 0.958037
[297]	valid_0's auc: 0.958082
[298]	valid_0's auc: 0.958091
[299]	valid_0's auc: 0.95816
[300]	valid_0's auc: 0.958199
[301]	valid_0's auc: 0.958261
[302]	valid_0's auc: 0.958338
[303]	valid_0's auc: 0.958396
[304]	valid_0's auc: 0.958459
[305]	valid_0's auc: 0.95855
[306]	valid_0's auc: 0.958607
[307]	valid_0's auc: 0.95863
[308]	valid_0's auc: 0.95868
[309]	valid_0's auc: 0.958711
[310]	valid_0's

[552]	valid_0's auc: 0.966845
[553]	valid_0's auc: 0.966885
[554]	valid_0's auc: 0.96688
[555]	valid_0's auc: 0.966925
[556]	valid_0's auc: 0.966941
[557]	valid_0's auc: 0.966954
[558]	valid_0's auc: 0.96698
[559]	valid_0's auc: 0.96698
[560]	valid_0's auc: 0.966973
[561]	valid_0's auc: 0.966972
[562]	valid_0's auc: 0.966969
[563]	valid_0's auc: 0.966976
[564]	valid_0's auc: 0.966968
[565]	valid_0's auc: 0.967001
[566]	valid_0's auc: 0.967029
[567]	valid_0's auc: 0.967032
[568]	valid_0's auc: 0.967062
[569]	valid_0's auc: 0.967075
[570]	valid_0's auc: 0.96711
[571]	valid_0's auc: 0.96712
[572]	valid_0's auc: 0.967148
[573]	valid_0's auc: 0.967151
[574]	valid_0's auc: 0.967159
[575]	valid_0's auc: 0.967169
[576]	valid_0's auc: 0.967197
[577]	valid_0's auc: 0.967215
[578]	valid_0's auc: 0.967252
[579]	valid_0's auc: 0.967263
[580]	valid_0's auc: 0.967278
[581]	valid_0's auc: 0.967289
[582]	valid_0's auc: 0.96729
[583]	valid_0's auc: 0.967305
[584]	valid_0's auc: 0.96735
[585]	valid_0's a

...

[1]	valid_0's auc: 0.865263
Training until validation scores don't improve for 20 rounds.
[2]	valid_0's auc: 0.889808
[3]	valid_0's auc: 0.897837
[4]	valid_0's auc: 0.898803
[5]	valid_0's auc: 0.901381
[6]	valid_0's auc: 0.903428
[7]	valid_0's auc: 0.910766
[8]	valid_0's auc: 0.91402
[9]	valid_0's auc: 0.919667
[10]	valid_0's auc: 0.923829
[11]	valid_0's auc: 0.925289
[12]	valid_0's auc: 0.928222
[13]	valid_0's auc: 0.930166
[14]	valid_0's auc: 0.931657
[15]	valid_0's auc: 0.934385
[16]	valid_0's auc: 0.93647
[17]	valid_0's auc: 0.938993
[18]	valid_0's auc: 0.939099
[19]	valid_0's auc: 0.940676
[20]	valid_0's auc: 0.94125
[21]	valid_0's auc: 0.941848
[22]	valid_0's auc: 0.942647
[23]	valid_0's auc: 0.944307
[24]	valid_0's auc: 0.944722
[25]	valid_0's auc: 0.945759
[26]	valid_0's auc: 0.946382
[27]	valid_0's auc: 0.947325
[28]	valid_0's auc: 0.948034
[29]	valid_0's auc: 0.948415
[30]	valid_0's auc: 0.9489
[31]	valid_0's auc: 0.94979
[32]	valid_0's auc: 0.950173
[33]	valid_0's auc: 0.951

[277]	valid_0's auc: 0.9764
[278]	valid_0's auc: 0.976423
[279]	valid_0's auc: 0.976444
[280]	valid_0's auc: 0.976487
[281]	valid_0's auc: 0.976475
[282]	valid_0's auc: 0.976527
[283]	valid_0's auc: 0.976567
[284]	valid_0's auc: 0.976574
[285]	valid_0's auc: 0.976587
[286]	valid_0's auc: 0.976518
[287]	valid_0's auc: 0.976505
[288]	valid_0's auc: 0.976447
[289]	valid_0's auc: 0.976433
[290]	valid_0's auc: 0.976401
[291]	valid_0's auc: 0.976407
[292]	valid_0's auc: 0.97641
[293]	valid_0's auc: 0.976419
[294]	valid_0's auc: 0.976428
[295]	valid_0's auc: 0.976441
[296]	valid_0's auc: 0.976489
[297]	valid_0's auc: 0.976484
[298]	valid_0's auc: 0.976495
[299]	valid_0's auc: 0.976547
[300]	valid_0's auc: 0.976562
[301]	valid_0's auc: 0.976542
[302]	valid_0's auc: 0.976567
[303]	valid_0's auc: 0.976613
[304]	valid_0's auc: 0.976608
[305]	valid_0's auc: 0.976555
[306]	valid_0's auc: 0.976587
[307]	valid_0's auc: 0.976613
[308]	valid_0's auc: 0.976646
[309]	valid_0's auc: 0.976698
[310]	valid_0

[551]	valid_0's auc: 0.978959
[552]	valid_0's auc: 0.978916
[553]	valid_0's auc: 0.978941
[554]	valid_0's auc: 0.978926
[555]	valid_0's auc: 0.978933
[556]	valid_0's auc: 0.978938
[557]	valid_0's auc: 0.978954
[558]	valid_0's auc: 0.978965
[559]	valid_0's auc: 0.978978
[560]	valid_0's auc: 0.978973
[561]	valid_0's auc: 0.978988
[562]	valid_0's auc: 0.979029
[563]	valid_0's auc: 0.97905
[564]	valid_0's auc: 0.979043
[565]	valid_0's auc: 0.979109
[566]	valid_0's auc: 0.979104
[567]	valid_0's auc: 0.979039
[568]	valid_0's auc: 0.978982
[569]	valid_0's auc: 0.978989
[570]	valid_0's auc: 0.979013
[571]	valid_0's auc: 0.979038
[572]	valid_0's auc: 0.979039
[573]	valid_0's auc: 0.979051
[574]	valid_0's auc: 0.979015
[575]	valid_0's auc: 0.978994
[576]	valid_0's auc: 0.97897
[577]	valid_0's auc: 0.978964
[578]	valid_0's auc: 0.978962
[579]	valid_0's auc: 0.97892
[580]	valid_0's auc: 0.978929
[581]	valid_0's auc: 0.978881
[582]	valid_0's auc: 0.97884
[583]	valid_0's auc: 0.978842
[584]	valid_0'

/home/kai/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:99: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/home/kai/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:104: UserWarning: Found `early_stopping_rounds` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[1]	valid_0's auc: 0.852861
Training until validation scores don't improve for 20 rounds.
[2]	valid_0's auc: 0.879711
[3]	valid_0's auc: 0.922845
[4]	valid_0's auc: 0.927734
[5]	valid_0's auc: 0.936892
[6]	valid_0's auc: 0.948202
[7]	valid_0's auc: 0.952814
[8]	valid_0's auc: 0.960303
[9]	valid_0's auc: 0.962455
[10]	valid_0's auc: 0.96464
[11]	valid_0's auc: 0.963914
[12]	valid_0's auc: 0.965073
[13]	valid_0's auc: 0.967091
[14]	valid_0's auc: 0.967419
[15]	valid_0's auc: 0.969698
[16]	valid_0's auc: 0.970437
[17]	valid_0's auc: 0.971909
[18]	valid_0's auc: 0.973181
[19]	valid_0's auc: 0.974068
[20]	valid_0's auc: 0.974821
[21]	valid_0's auc: 0.974278
[22]	valid_0's auc: 0.974537
[23]	valid_0's auc: 0.975017
[24]	valid_0's auc: 0.975203
[25]	valid_0's auc: 0.976093
[26]	valid_0's auc: 0.976247
[27]	valid_0's auc: 0.976257
[28]	valid_0's auc: 0.97658
[29]	valid_0's auc: 0.977436
[30]	valid_0's auc: 0.977317
[31]	valid_0's auc: 0.977666
[32]	valid_0's auc: 0.977293
[33]	valid_0's auc: 0

In [60]:
for col in label_cols:
    print(col)
    add= None
    params = base_params.copy()
    params_get, index_get = get_parameters_best(df_lgb, col, addition=add)
    print('------------------------')

toxic
bagging_fraction          0.7
combination_id              1
feature_fraction            1
fp                  0.0421443
label                   toxic
lambda_l1                   1
lambda_l2                   1
learning_rate            0.05
max_depth                   4
num_iterations           8000
num_leaves                 31
precision            0.680242
roc                  0.973945
tp                   0.852486
trained                     Y
is_unbalance             True
best_round                  0
Name: 3672, dtype: object
------------------------
severe_toxic
bagging_fraction             0.9
combination_id                 2
feature_fraction             0.6
fp                     0.0289846
label               severe_toxic
lambda_l1                    0.5
lambda_l2                      0
learning_rate                0.1
max_depth                      3
num_iterations              8000
num_leaves                    61
precision               0.234114
roc                     

In [11]:
aa.predict(val_set[col])

NameError: name 'aa' is not defined

In [44]:
print(aa.best_iteration_)

509


In [19]:
b